# **GPU required. Please open this notebook in the Colab**

<a href="https://colab.research.google.com/github/apssouza22/computer-vision/blob/master/ze-image-animation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Demo for paper "First Order Motion Model for Image Animation"

![alt text](https://aliaksandrsiarohin.github.io/first-order-model-website/vox-teaser.gif)

Check out the website of the model https://aliaksandrsiarohin.github.io/first-order-model-website/

**Downloading the model and required assets**

In [1]:
import requests

def download(url, save_path, chunk_size=128):
	r = requests.get(url, stream=True)
	with open(save_path, 'wb') as fd:
		for chunk in r.iter_content(chunk_size=chunk_size):
			fd.write(chunk)
   

download("http://apssouza.com.br/downloads/vox-cpk.pth.tar", "vox-cpk.pth.tar")
download("http://apssouza.com.br/downloads/my-video-no-audio.mov", "my-video-no-audio.mov")
download("http://apssouza.com.br/downloads/elon-musk.png", "elon-musk.png")


**Clone repository**

In [2]:
!git clone https://github.com/AliaksandrSiarohin/first-order-model

Cloning into 'first-order-model'...
remote: Enumerating objects: 236, done.
remote: Total 236 (delta 0), reused 0 (delta 0), pack-reused 236
Receiving objects: 100% (236/236), 71.46 MiB | 10.80 MiB/s, done.
Resolving deltas: 100% (117/117), done.


In [3]:
cd first-order-model

/content/first-order-model


In [4]:
import imageio
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from skimage.transform import resize
from IPython.display import HTML
import warnings
warnings.filterwarnings("ignore")


**Create a model and load checkpoints**

In [5]:

from demo import load_checkpoints
generator, kp_detector = load_checkpoints(
    config_path='config/vox-256.yaml', 
    checkpoint_path='../vox-cpk.pth.tar',
)

**Prepare the driving video**

In [ ]:
!ffmpeg -i ../my-video-no-audio.mov -vf scale=700:700 ../my-video-out.mp4

!ffmpeg -i ../my-video-out.mp4 -filter:v "crop=600:600:500:50" ../my-video-crop.mp4



In [1]:
# Help function to display the video
def display(source, driving, generated=None):
    fig = plt.figure(figsize=(8 + 4 * (generated is not None), 6))

    ims = []
    for i in range(len(driving)):
        cols = [source]
        cols.append(driving[i])
        if generated is not None:
            cols.append(generated[i])
        im = plt.imshow(np.concatenate(cols, axis=1), animated=True)
        plt.axis('off')
        ims.append([im])

    ani = animation.ArtistAnimation(fig, ims, interval=50, repeat_delay=1000)
    plt.close()
    return ani

**Perform image animation**

In [ ]:
from demo import make_animation
from skimage import img_as_ubyte

source_image = imageio.imread('../elon-musk.png')
driving_video = imageio.mimread('../my-video-crop.mp4', memtest=False)

source_image = resize(source_image, (256, 256))[..., :3]
driving_video = [resize(frame, (256, 256))[..., :3] for frame in driving_video]

predictions = make_animation(source_image, driving_video, generator, kp_detector, relative=True)

#save resulting video
imageio.mimsave('../generated.mp4', [img_as_ubyte(frame) for frame in predictions])
#video can be downloaded from /content folder

HTML(display(source_image, driving_video, predictions).to_html5_video())